In [30]:
import os
import ast
import random
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

class DataLoader:
    def __init__(self, base_path="gerest"):
        self.base_path = base_path

    def load_data(self, name, data_type, cv=False, seed=42, target="asqp"):
        data_types = ["train", "test", "dev"] if data_type == "all" else [data_type]
        data = []

        for dtype in data_types:
            dataset_path = os.path.join(f"gerest/{dtype}.txt")
            if not os.path.exists(dataset_path):
                raise FileNotFoundError(f"Dataset file {dataset_path} not found.")
            
            with open(dataset_path, 'r', encoding='utf-8') as file:
                lines = file.readlines()
            
            for idx, line in enumerate(lines):
                try:
                    text, aspects_str = line.split("####")
                    aspects = ast.literal_eval(aspects_str.strip())
                    aspect_list = []

                    for aspect in aspects:
                        aspect_dict = {
                            "aspect_term": aspect[0],
                            "aspect_category": aspect[1],
                            "polarity": aspect[2]
                        }
                        # Add 'opinion_term' only if target is 'asqp'
                        if target == "asqp":
                            aspect_dict["opinion_term"] = aspect[3]
                        aspect_list.append(aspect_dict)

                    if len(aspects) > 0:
                        data.append({
                            "id": f"{idx}_{name}_{dtype}",
                            "text": text.strip(),
                            "aspects": aspect_list,
                            "tuple_list": [tuple(aspect) for aspect in aspects]
                        })
                except ValueError as e:
                    print(f"Skipping line {idx} in {dtype} due to ValueError: {e}")
                    continue
        
        if cv:
            return self.random_cross_validation_split(data, seed)
        
        return data




    def random_cross_validation_split(self, data, seed=42):
        categories = [[el["aspect_category"] for el in an["aspects"]] for an in data]

        mlb = MultiLabelBinarizer()
        Y = mlb.fit_transform(categories)

        n_splits = 5
        mskf = MultilabelStratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    
        splits = []
        for train_index, test_index in mskf.split(np.zeros(len(Y)), Y):
           splits.append([data[i] for i in test_index])
    
    
        return splits
    
dataloader = DataLoader()

data = dataloader.load_data("rest16", "train", target="asqp")

In [34]:
shown_sentences = set()  # Set zum Speichern bereits ausgegebener Sätze

for entry in data:
    for aspect in entry['aspects']:
        # Prüfen, ob die Kategorie "airline price" ist und der Satz noch nicht gezeigt wurde
        if aspect['aspect_category'] == "service general" and entry['text'] not in shown_sentences:
            print(aspect["opinion_term"], "|", aspect["aspect_term"] , "|" , entry['text'])
            shown_sentences.add(entry['text'])  # Satz als gezeigt markieren

freundliches | Personal | Sehr freundliches Personal, mit dem italienischen Outfit ein Stück LOC in LOC.
netten | Bedienung | Auf die Bitte die Tische zusammen schieben zu können wurde uns von einer der sehr netten Bedienung gesagt - Chefin hat das Verboten.
geht es mir mittlerweile auf den Keks | Warterei | Denn irgendwie geht es mir mittlerweile auf den Keks diese Warterei.
freundlich | Service | Service war freundlich und schnell.
nett | Mitarbeiter | Das Essen ist super lecker und die Mitarbeiter total nett!
haben über eine halbe Stunde gewartet | NULL | Jedoch wurde die angekündigte Fertigstellungszeit weit überschritten und wir haben über eine halbe Stunde gewartet.
top | Service | Mit Sicherheit eines der besten Steak- Häuser in LOC, wir haben uns komplett wohl gefühlt, Service war top top top!
unfreundlich | Bedienung | Bedienung unfreundlich mit verdreckten Oberteil - sorry unappetitlich hoch drei .
ein Glas mit Lippenstift | NULL | Nachdem wir nach einiger Zeit unser Getränk 

In [32]:
data[0]

{'id': '0_rest16_train',
 'text': 'Sehr freundliches Personal, mit dem italienischen Outfit ein Stück LOC in LOC.',
 'aspects': [{'aspect_term': 'Personal',
   'aspect_category': 'service general',
   'polarity': 'positive',
   'opinion_term': 'freundliches'}],
 'tuple_list': [('Personal', 'service general', 'positive', 'freundliches')]}